In [ ]:
import numpy as np

In [ ]:
import numpy as np

def convolve_via_ft_on_grid(
    fhat, ghat,
    *,
    N: int = 2**14,
    dt: float = 1e-2,
    # If your analytic transforms use angular frequency ω (a.k.a. k) in rad/unit:
    use_angular: bool = True,
):
    """
    Compute h(t) = (f*g)(t) from analytic Fourier transforms:
        ĥ(k) = f̂(k) ĝ(k),   h(t) = (1/2π) ∫ ĥ(k) e^{i k t} dk
    using an FFT/IFFT on a uniform grid.

    Parameters
    ----------
    fhat, ghat : callables
        Functions fhat(k) and ghat(k) returning complex values.
        k is either angular frequency (rad/unit) if use_angular=True,
        or cycles/unit if use_angular=False.
    N : int
        Number of time samples (power of 2 is convenient).
    dt : float
        Time spacing (sets the frequency grid via dk = 2π/(N dt) for angular).
    use_angular : bool
        True if your transform variable is angular k=ω (rad/unit).
        False if your transform variable is ν (cycles/unit) with exp(±2π i ν t).

    Returns
    -------
    t : (N,) ndarray
        Time grid (centered at 0).
    h : (N,) ndarray
        Convolution values h(t) on that grid.
    k : (N,) ndarray
        Frequency grid (k or ν matching use_angular).
    hhat : (N,) ndarray
        Sampled product transform on the frequency grid.
    """
    # Time grid centered at 0 for nicer looking curves
    t = (np.arange(N) - N // 2) * dt

    # Frequency grid that matches numpy's FFT ordering (0, +, -, ...)
    nu = np.fft.fftfreq(N, d=dt)  # cycles/unit
    if use_angular:
        k = 2 * np.pi * nu         # rad/unit
        dk = 2 * np.pi / (N * dt)
    else:
        k = nu
        dk = 1.0 / (N * dt)

    # Sample the product transform in FFT ordering
    hhat = fhat(k) * ghat(k)

    # Inverse transform to time domain.
    # np.fft.ifft implements (1/N) Σ H_k e^{+2π i k n/N}.
    # To approximate the continuous inverse:
    #   h(t) = (1/2π) ∫ ĥ(k) e^{i k t} dk  (angular)
    # or
    #   h(t) = ∫ ĥ(ν) e^{2π i ν t} dν      (cycles)
    h_time = np.fft.ifft(hhat)

    if use_angular:
        # Map discrete sum to integral: (dk / 2π) Σ ĥ(k) e^{i k t}
        h_time *= (N * dk) / (2 * np.pi)  # because ifft already has 1/N
    else:
        # cycles convention: h(t) ≈ dν Σ ĥ(ν) e^{2π i ν t}
        h_time *= (N * dk)                # dk == dν

    # Center result to match centered t grid
    h_time = np.fft.fftshift(h_time)

    return t, h_time, k, hhat


# ----------------------------
# Example usage with a Gaussian in k-space
# ----------------------------
def make_gaussian_filter_kspace(sigma_k: float, *, use_angular: bool = True):
    """
    Gaussian low-pass in frequency domain.
    If you're literally multiplying by exp(-k^2/(2σ_k^2)), sigma_k is in k-units.
    """
    def ghat(k):
        return np.exp(-0.5 * (k / sigma_k) ** 2)
    return ghat


# Suppose you already have analytic fhat(k) and you're "convolving with a Gaussian"
# by multiplying in k-space: ĥ(k) = f̂(k) * exp(-k^2/(2σ_k^2))
# Then just set ghat to that Gaussian and run convolve_via_ft_on_grid.

# Example dummy fhat:
def fhat_example(k):
    # Example: Fourier transform of a Lorentzian in time ~ exp(-a|t|)
    a = 1.0
    return 2 * a / (a**2 + k**2)  # (up to convention factors)

sigma_k = 5.0
ghat = make_gaussian_filter_kspace(sigma_k)

# Pick dt and N so that k_max is several sigma_k:
# For angular k, k_max ≈ π/dt. So choose dt <= π/(m*sigma_k), e.g. m=6.
m = 6
dt = np.pi / (m * sigma_k)
N = 2**14  # increase N to extend the time window T = N*dt

t, h, k, hhat = convolve_via_ft_on_grid(fhat_example, ghat, N=N, dt=dt, use_angular=True)

# h should be (approximately) real if your setup is physically real-valued
h_real = h.real
